In [3]:
import numpy as np
import pandas as pd
import os

In [4]:
# import datasets

path = r'/Users/polusa/Library/Mobile Documents/com~apple~CloudDocs/my_DA_2024/CareerFoundry_Data_Analytics_Bootcamp/4-Python_Fundamentals_for_DA/04-2024_Instacart_Basket_Analysis/02-Data'

df_ords = pd.read_csv(os.path.join(path, '01-Raw_Data', 'orders.csv'), index_col=False)
df_prods = pd.read_csv(os.path.join(path, '01-Raw_Data', 'products.csv'), index_col=False)
df_dep = pd.read_csv(os.path.join(path, '01-Raw_Data', 'departments.csv'), index_col=False)

Let's remove the column `eval_set' from the dataframe since it is not needed

In [33]:
df_ords = df_ords.drop(columns=['eval_set'])
df_ords

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0
...,...,...,...,...,...,...
3421078,2266710,206209,10,5,18,29.0
3421079,1854736,206209,11,4,10,30.0
3421080,626363,206209,12,1,12,18.0
3421081,2977660,206209,13,1,12,7.0


Let's check the frequency of values in the `days_since_prior_order` column:

In [34]:
df_ords['days_since_prior_order'].value_counts(dropna=False)

days_since_prior_order
30.0    369323
7.0     320608
6.0     240013
4.0     221696
3.0     217005
5.0     214503
NaN     206209
2.0     193206
8.0     181717
1.0     145247
9.0     118188
14.0    100230
10.0     95186
13.0     83214
11.0     80970
12.0     76146
0.0      67755
15.0     66579
16.0     46941
21.0     45470
17.0     39245
20.0     38527
18.0     35881
19.0     34384
22.0     32012
28.0     26777
23.0     23885
27.0     22013
24.0     20712
25.0     19234
29.0     19191
26.0     19016
Name: count, dtype: int64

Let's now rename the column `order_dow` to a more meaningful `orders_day_of_week`:

In [35]:
ords_new_col = {
    'order_dow': 'orders_day_of_week'
}
df_ords.rename(columns = ords_new_col, inplace = True)
df_ords.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0


Let's now calculate some descriptive statistics for only the columns we know we need.  
Statistics about the `order_id` or `user_id` don't make much sense after all.  

To do so, we use the `describe()` method, however to exclude the columns we don't need we need to change their data type to `string` since the describe only caluclates the statistics on numerical (quantitative) columns. 

In [36]:
df_ords['order_id'] = df_ords['order_id'].astype('str')

In [37]:
df_ords['order_id'].dtypes # 'O' stands for object, the pandas way to say string

dtype('O')

__Transposing__ refers to turning your dataframe’s rows into columns, and vice versa.  
This is also known as changing your data from _“wide format”_ into _“long format”_.

In [38]:
df_dep.head()

,department_id,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,department,frozen,other,bakery,produce,alcohol,international,beverages,pets,dry goods pasta,...,meat seafood,pantry,breakfast,canned goods,dairy eggs,household,babies,snacks,deli,missing


In [39]:
df_dep_t = df_dep.T

In [40]:
df_dep_t

,0
department_id,department
1,frozen
2,other
3,bakery
4,produce
5,alcohol
6,international
7,beverages
8,pets
9,dry goods pasta


Let’s start by adding something to your dataframe: an __index__.  
Remember, this is the very first column on the left side of the dataframe that shows you the consecutive number of rows.  
You can add an index to a dataframe that doesn’t have one with the following function:

In [41]:
df_dep_t.reset_index() # remember this line doesn't save the new df as it is not assigned to any variable

,index,0
0,department_id,department
1,1,frozen
2,2,other
3,3,bakery
4,4,produce
5,5,alcohol
6,6,international
7,7,beverages
8,8,pets
9,9,dry goods pasta


Now that you have an index column, let’s create a new header for your dataframe.  
First, you need to tell Python to take the row under the current header (the first row in the dataframe),  
use that data to create a new header, and get rid of the current header.

In [42]:
new_header = df_dep_t.iloc[0]

Now that you’ve made a copy of your first row, you need to delete it. Otherwise, when you add the first row back in as your header, you’ll have two of them: one as a header and one as a row. That’s not what you want! As such, let’s create a new dataframe that only copies over rows beyond the first row. The command for this is as follows:

In [43]:
df_dep_t_new = df_dep_t[1:]

In [44]:
df_dep_t_new.head()

,0
1,frozen
2,other
3,bakery
4,produce
5,alcohol


All that’s left to do now is set your new_header variable as the new header for your `df_dep_t_new` dataframe.

In [45]:
df_dep_t_new.columns = new_header

In [46]:
df_dep_t_new

department_id,department
1,frozen
2,other
3,bakery
4,produce
5,alcohol
6,international
7,beverages
8,pets
9,dry goods pasta
10,bulk


Let’s use pandas and create a __data dictionary__ that contains meanings for the values in the "department_id" column of the `df_prods` data frame.

In [47]:
df_dep_t_new

department_id,department
1,frozen
2,other
3,bakery
4,produce
5,alcohol
6,international
7,beverages
8,pets
9,dry goods pasta
10,bulk


In [48]:
# turn your df_dep_t_new dataframe into a dictionary
data_dict = df_dep_t_new.to_dict('index')
data_dict

{'1': {'department': 'frozen'},
 '2': {'department': 'other'},
 '3': {'department': 'bakery'},
 '4': {'department': 'produce'},
 '5': {'department': 'alcohol'},
 '6': {'department': 'international'},
 '7': {'department': 'beverages'},
 '8': {'department': 'pets'},
 '9': {'department': 'dry goods pasta'},
 '10': {'department': 'bulk'},
 '11': {'department': 'personal care'},
 '12': {'department': 'meat seafood'},
 '13': {'department': 'pantry'},
 '14': {'department': 'breakfast'},
 '15': {'department': 'canned goods'},
 '16': {'department': 'dairy eggs'},
 '17': {'department': 'household'},
 '18': {'department': 'babies'},
 '19': {'department': 'snacks'},
 '20': {'department': 'deli'},
 '21': {'department': 'missing'}}

In [5]:
df_prods.head()

,product_id,product_name,aisle_id,department_id,prices
0,1,Chocolate Sandwich Cookies,61,19,5.8
1,2,All-Seasons Salt,104,13,9.3
2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,5,Green Chile Anytime Sauce,5,13,4.3


In [50]:
#data_dict['19']
data_dict.get('19', 'not found')

{'department': 'snacks'}

#### Subsetting (similar to conditional slicing in numpy)
Subsetting refers to creating a smaller data set from a whole data set (in this case, `df_prods`) based on a particular filter.

In [51]:
df_snacks = df_prods[ df_prods['department_id'] == 19]
df_snacks.head()

,product_id,product_name,aisle_id,department_id,prices
0,1,Chocolate Sandwich Cookies,61,19,5.8
15,16,Mint Chocolate Flavored Syrup,103,19,5.2
24,25,Salted Caramel Lean Protein & Fiber Bar,3,19,1.9
31,32,Nacho Cheese White Bean Chips,107,19,4.9
40,41,Organic Sourdough Einkorn Crackers Rosemary,78,19,6.5


In this second version, rather than using an equals sign, you’re telling the loc function to look into a list: isin([19]). This is handy because you could use more than one value here if you wanted to check multiple "department_ids" at the same time (isin([17,18,19])).

In [52]:
df_prods.loc[df_prods['department_id'].isin([19])]

,product_id,product_name,aisle_id,department_id,prices
0,1,Chocolate Sandwich Cookies,61,19,5.8
15,16,Mint Chocolate Flavored Syrup,103,19,5.2
24,25,Salted Caramel Lean Protein & Fiber Bar,3,19,1.9
31,32,Nacho Cheese White Bean Chips,107,19,4.9
40,41,Organic Sourdough Einkorn Crackers Rosemary,78,19,6.5
...,...,...,...,...,...
49666,49662,Bacon Cheddar Pretzel Pieces,107,19,3.6
49669,49665,Super Dark Coconut Ash & Banana Chocolate Bar,45,19,6.9
49670,49666,Ginger Snaps Snacking Cookies,61,19,5.2
49675,49671,Milk Chocolate Drops,45,19,3.0


# Task 4.4

#### 2. Find another identifier variable in the `df_ords` dataframe that doesn’t need to be included in your analysis as a numeric variable and change it to a suitable format.

In [58]:
df_ords.columns

Index(['order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_prior_order'],
      dtype='object')

In [60]:
df_ords['user_id'] = df_ords['user_id'].astype('str')

Check the data type of columns `user_id` after casting it

In [61]:
df_ords['user_id'].dtype

dtype('O')

#### 3. Look for a variable in your df_ords dataframe with an unintuitive name and change its name without overwriting the dataframe.

In [65]:
df_ords.rename(columns={'days_since_prior_order': 'days_since_last_order'}, inplace=True)

In [66]:
df_ords.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order
0,2539329,1,1,2,8,NaN
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0


#### 4. Your client wants to know what the busiest hour is for placing orders. Find the frequency of the corresponding variable and share your findings.  

10 o'clock is the hour with more orders

In [75]:
df_ords.loc[:, 'order_hour_of_day'].value_counts(dropna=False).sort_values(ascending=False)

order_hour_of_day
10    288418
11    284728
15    283639
14    283042
13    277999
12    272841
16    272553
9     257812
17    228795
18    182912
8     178201
19    140569
20    104292
7      91868
21     78109
22     61468
23     40043
6      30529
0      22758
1      12398
5       9569
2       7539
4       5527
3       5474
Name: count, dtype: int64

#### 5. Determine the meaning behind a value of 4 in the "department_id" column within the df_prods dataframe using a data dictionary.


In [6]:
data_dict.get('4', 'Not Found')['department']

NameError: name 'data_dict' is not defined

#### 6. The sales team in your client’s organization wants to know more about breakfast item sales. Create a subset containing only the required information.

In [85]:
data_dict

{'1': {'department': 'frozen'},
 '2': {'department': 'other'},
 '3': {'department': 'bakery'},
 '4': {'department': 'produce'},
 '5': {'department': 'alcohol'},
 '6': {'department': 'international'},
 '7': {'department': 'beverages'},
 '8': {'department': 'pets'},
 '9': {'department': 'dry goods pasta'},
 '10': {'department': 'bulk'},
 '11': {'department': 'personal care'},
 '12': {'department': 'meat seafood'},
 '13': {'department': 'pantry'},
 '14': {'department': 'breakfast'},
 '15': {'department': 'canned goods'},
 '16': {'department': 'dairy eggs'},
 '17': {'department': 'household'},
 '18': {'department': 'babies'},
 '19': {'department': 'snacks'},
 '20': {'department': 'deli'},
 '21': {'department': 'missing'}}

In [7]:
# loop through data_dict to find the corresponding id for 'breakfast' department
for i in data_dict.items():
    if i[1]['department'].lower() == 'breakfast':
        breakfast_dep_id = i[0]

# subsetting to filter out all item that are not in the breakfast department
df_prods_breakfast = df_prods[df_prods['department_id'] == int(breakfast_dep_id)]
df_prods_breakfast.head()

NameError: name 'data_dict' is not defined

#### 7. They’d also like to see details about products that customers might use to throw dinner parties. Your task is to find all observations from the entire dataframe that include items from the following departments: alcohol, deli, beverages, and meat/seafood. You’ll need to present this subset to your client.

In [123]:
# loop thorugh data_dict to find 'department id' of items needed and save it in a list
dep_items_id = []
for i in data_dict.items():
    if i[1]['department'].lower() in ['alcohol', 'deli', 'beverages', 'meat seafood']:
        dep_items_id.append(int(i[0]))

# subsetting the dataframe using the list of ids previously created
df_prods_dinner_party = df_prods[ df_prods['department_id'].isin(dep_items_id)]
df_prods_dinner_party.head()

,product_id,product_name,aisle_id,department_id,prices
2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
6,7,Pure Coconut Water With Orange,98,7,4.4
9,10,Sparkling Orange Juice & Prickly Pear Beverage,115,7,8.4
10,11,Peach Mango Juice,31,7,2.8
16,17,Rendered Duck Fat,35,12,17.1


#### 8. It’s important that you keep track of total counts in your dataframes. How many rows does the last dataframe you created have?

In [126]:
print(f'Number of rows: {df_prods_dinner_party.shape[0]}')

Number of rows: 7650


#### 9. Someone from the data engineers team in Instacart thinks they’ve spotted something strange about the customer with a "user_id" of “1.” Extract all the information you can about this user.

In [130]:
df_ords_usr1 = df_ords[ df_ords['user_id'] == '1']
df_ords_usr1

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order
0,2539329,1,1,2,8,NaN
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0
5,3367565,1,6,2,7,19.0
6,550135,1,7,1,9,20.0
7,3108588,1,8,1,14,14.0
8,2295261,1,9,1,16,0.0
9,2550362,1,10,4,8,30.0


#### 10. You also need to provide some details about this user’s behavior. What basic stats can you provide based on the information you have?

In [132]:
df_ords_usr1.describe()

,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order
count,11.000000,11.000000,11.000000,10.000000
mean,6.000000,2.636364,10.090909,19.000000
std,3.316625,1.286291,3.477198,9.030811
min,1.000000,1.000000,7.000000,0.000000
25%,3.500000,1.500000,7.500000,14.250000
50%,6.000000,3.000000,8.000000,19.500000
75%,8.500000,4.000000,13.000000,26.250000
max,11.000000,4.000000,16.000000,30.000000


We can see that they placed a total of 11 orders that are at maximum 30 days apart, and even orders on consecutive days.  
On average he places an order around 10am between tuesday and wednesday.  
We can also check for what is the most common hour they place and order and what the most common day: 

In [147]:
# print the mode of the two columns (orders_day_of_week, order_hour_of_day)

print(df_ords_usr1['orders_day_of_week'].mode(),'\n')
print(df_ords_usr1['order_hour_of_day'].mode())

0    4
Name: orders_day_of_week, dtype: int64 

0    7
1    8
Name: order_hour_of_day, dtype: int64


As we can see, the most common day they places an order was on day 4 (Thursday) and the two most common hours were 7am, and 8am (bimodal).

In [148]:
df_dep_t_new


department_id,department
1,frozen
2,other
3,bakery
4,produce
5,alcohol
6,international
7,beverages
8,pets
9,dry goods pasta
10,bulk


In [150]:
# export department dataframe after fixing it

df_dep_t_new.to_csv(os.path.join(path, '02-Prepared_Data', 'departments_wrangled.csv'))

In [8]:
df_prods.shape

(49693, 5)